<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# AzureML Pipeline, AutoML, AKS Deployment for Sentence Similarity

This notebook demonstrates how to use [Azure Machine Learning](https://azure.microsoft.com/en-us/services/machine-learning-service/
) pipelines and Automated Machine Learning ([AutoML](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml
)) to streamline the creation of a machine learning workflow for predicting sentence similarity. The pipeline contains two steps:   
1. PythonScriptStep: embeds sentences using a popular sentence embedding model, Google Universal Sentence Encoder
2. AutoMLStep: demonstrates how to use Automated Machine Learning (AutoML) to automate model selection for predicting sentence similarity (regression)

After creating the pipeline, the notebook demonstrates the deployment of our sentence similarity model using Azure Kubernetes Service ([AKS](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes
)).

This notebook showcases how to use the following AzureML features:  
- AzureML Pipelines (PythonScriptStep and AutoMLStep)
- Automated Machine Learning
- AmlCompute
- Datastore
- Logging

## Table of Contents
1. [Introduction](#1.-Introduction)  
    * 1.1 [What are AzureML Pipelines?](#1.1-What-are-AzureML-Pipelines?)  
    * 1.2 [What is Azure AutoML?](#1.2-What-is-Azure-AutoML?)  
    * 1.3 [Modeling Problem](#1.3-Modeling-Problem)  
2. [Data Preparation](#2.-Data-Preparation)  
3. [AzureML Setup](#3.-AzureML-Setup)  
    * 3.1 [Link to or create a `Workspace`](#3.1-Link-to-or-create-a-Workspace)  
    * 3.2 [Set up an `Experiment` and Logging](#3.2-Set-up-an-Experiment-and-Logging)  
    * 3.3 [Link `AmlCompute` compute target](#3.3-Link-AmlCompute-compute-target)  
    * 3.4 [Upload data to `Datastore`](#3.4-Upload-data-to-Datastore)  
4. [Create AzureML Pipeline](#4.-Create-AzureML-Pipeline)  
    * 4.1 [Set up run configuration file](#4.1-Set-up-run-configuration-file)  
    * 4.2 [PythonScriptStep](#4.2-PythonScriptStep)  
        * 4.2.1 [Define python script to run](#4.2.1-Define-python-script-to-run)
        * 4.2.2 [Create PipelineData object](#4.2.2-Create-PipelineData-object)
        * 4.2.3 [Create PythonScriptStep](#4.2.3-Create-PythonScriptStep)
    * 4.3 [AutoMLStep](#4.3-AutoMLStep)
        * 4.3.1 [Define get_data script to load data](#4.3.1-Define-get_data-script-to-load-data)
        * 4.3.2 [Create AutoMLConfig object](#4.3.2-Create-AutoMLConfig-object)
        * 4.3.3 [Create AutoMLStep](#4.3.3-Create-AutoMLStep)    
5. [Run Pipeline](#5.-Run-Pipeline)  
6. [Deploy Sentence Similarity Model](#6.-Deploy-Sentence-Similarity-Model)
    * 6.1 [Register/Retrieve AutoML and Google Universal Sentence Encoder Models for Deployment](#6.1-Register/Retrieve-AutoML-and-Google-Universal-Sentence-Encoder-Models-for-Deployment)  
    * 6.2 [Create Scoring Script](#6.2-Create-Scoring-Script)
    * 6.3 [Create a YAML File for the Environment](#6.3-Create-a-YAML-File-for-the-Environment)   
    * 6.4 [Image Creation](#6.4-Image-Creation) 
    * 6.5 [Provision the AKS Cluster](#6.5-Provision-the-AKS-Cluster)   
    * 6.6 [Deploy the image as a Web Service to Azure Kubernetes Service](#6.6-Deploy-the-image-as-a-Web-Service-to-Azure-Kubernetes-Service)  
    * 6.7 [Test Deployed Model](#6.7-Test-Deployed-Webservice)  
    


## 1. Introduction

### 1.1 What are AzureML Pipelines?

[AzureML Pipelines](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines) define reusable machine learning workflows that can be used as a template for your machine learning scenarios. Pipelines allow you to optimize your workflow and spend time on machine learning rather than infrastructure. A Pipeline is defined by a series of steps; the following steps are available: AdlaStep, AutoMLStep, AzureBatchStep, DataTransferStep, DatabricksStep, EstimatorStep, HyperDriveStep, ModuleStep, MpiStep, and PythonScriptStep (see [here](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/?view=azure-ml-py) for details of each step). When the pipeline is run, cached results are used for all steps that have not changed, optimizing the run time. Data sources and intermediate data can be used across multiple steps in a pipeline, saving time and resources. Below we see an example of an AzureML pipeline.

![](https://nlpbp.blob.core.windows.net/images/pipelines.png)

### 1.2 What is Azure AutoML?

Automated machine learning ([AutoML](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml)) is a capability of Microsoft's [Azure Machine Learning service](https://azure.microsoft.com/en-us/services/machine-learning-service/
). The goal of AutoML is to improve the productivity of data scientists and democratize AI by allowing for the rapid development and deployment of machine learning models. To acheive this goal, AutoML automates the process of selecting a ML model and tuning the model. All the user is required to provide is a dataset (suitable for a classification, regression, or time-series forecasting problem) and a metric to optimize in choosing the model and hyperparameters. The user is also given the ability to set time and cost constraints for the model selection and tuning.

![](automl.png)

The AutoML model selection and tuning process can be easily tracked through the Azure portal or directly in python notebooks through the use of widgets. AutoML quickly selects a high quality machine learning model tailored for your prediction problem. In this notebook, we walk through the steps of preparing data, setting up an AutoML experiment, and evaluating the results of our best model. More information about running AutoML experiments in Python can be found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train). 

### 1.3 Modeling Problem

The regression problem we will demonstrate is predicting sentence similarity scores on the STS Benchmark dataset. The [STS Benchmark dataset](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark#STS_benchmark_dataset_and_companion_dataset) contains a selection of English datasets that were used in Semantic Textual Similarity (STS) tasks 2012-2017. The dataset contains 8,628 sentence pairs with a human-labeled integer representing the sentences' similarity (ranging from 0, for no meaning overlap, to 5, meaning equivalence).

For each sentence in the sentence pair, we will use Google's pretrained Universal Sentence Encoder (details provided below) to generate a $512$-dimensional embedding. Both embeddings in the sentence pair will be concatenated and the resulting $1024$-dimensional vector will be used as features in our regression problem. Our target variable is the sentence similarity score.

In [1]:
#1
# Set the environment path to find NLP
import sys

sys.path.append("../../")
import time
import logging
import csv
import os
import pandas as pd
import shutil
import numpy as np
import torch
import sys
from scipy.stats import pearsonr
from scipy.spatial import distance
from sklearn.externals import joblib
import json

# Import utils
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset import stsbenchmark
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
from utils_nlp.common.timer import Timer

# Google Universal Sentence Encoder loader
import tensorflow_hub as hub

# AzureML packages
import azureml as aml
import logging
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)
from azureml.core import Datastore, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.image import ContainerImage
from azureml.core.model import Model
from azureml.train.automl import AutoMLStep, AutoMLStepRun, AutoMLConfig
from azureml.pipeline.core import Pipeline, PipelineData, TrainingOutput
from azureml.pipeline.steps import PythonScriptStep
from azureml.data.data_reference import DataReference
from azureml.widgets import RunDetails

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", aml.core.VERSION)
print("Pandas version: {}".format(pd.__version__))

WARNING - Some hub symbols are not available because TensorFlow version is less than 1.14


Turning diagnostics collection on. 
System version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Azure ML SDK Version: 1.0.43
Pandas version: 0.23.4


In [2]:
#2
BASE_DATA_PATH = "../../data"

In [3]:
#3
automl_settings = {
    "task": "regression",  # type of task: classification, regression or forecasting
    "iteration_timeout_minutes": 15,  # How long each iteration can take before moving on
    "iterations": 50,  # Number of algorithm options to try
    "primary_metric": "spearman_correlation",  # Metric to optimize
    "preprocess": True,  # Whether dataset preprocessing should be applied
    "verbosity": logging.INFO,
}

# 2. Data Preparation

**STS Benchmark Dataset**

As described above, the STS Benchmark dataset contains 8.6K sentence pairs along with a human-annotated score for how similar the two sentences are. We will load the training, development (validation), and test sets provided by STS Benchmark and preprocess the data (lowercase the text, drop irrelevant columns, and rename the remaining columns) using the utils contained in this repo. Each dataset will ultimately have three columns: _sentence1_ and _sentence2_ which contain the text of the sentences in the sentence pair, and _score_ which contains the human-annotated similarity score of the sentence pair.

In [4]:
#4
# Load in the raw datasets as pandas dataframes
train_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")
dev_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="dev")
test_raw = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="test")

100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 253KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 267KB/s]


Data downloaded to ../../data\raw\stsbenchmark


100%|██████████████████████████████████████████████████████████████████████████████████| 401/401 [00:01<00:00, 237KB/s]


Data downloaded to ../../data\raw\stsbenchmark


In [5]:
#5
# Clean each dataset by lowercasing text, removing irrelevant columns,
# and renaming the remaining columns
train_clean = stsbenchmark.clean_sts(train_raw)
dev_clean = stsbenchmark.clean_sts(dev_raw)
test_clean = stsbenchmark.clean_sts(test_raw)

In [6]:
#6
# Convert all text to lowercase
train = to_lowercase(train_clean)
dev = to_lowercase(dev_clean)
test = to_lowercase(test_clean)

In [7]:
#7
print("Training set has {} sentences".format(len(train)))
print("Development set has {} sentences".format(len(dev)))
print("Testing set has {} sentences".format(len(test)))

Training set has 5749 sentences
Development set has 1500 sentences
Testing set has 1379 sentences


In [8]:
#8
train.head()

,score,sentence1,sentence2
0,5.00,a plane is taking off.,an air plane is taking off.
1,3.80,a man is playing a large flute.,a man is playing a flute.
2,3.80,a man is spreading shreded cheese on a pizza.,a man is spreading shredded cheese on an uncoo...
3,2.60,three men are playing chess.,two men are playing chess.
4,4.25,a man is playing the cello.,a man seated is playing the cello.


In [9]:
#9
# Save the cleaned data
if not os.path.isdir("data"):
    os.mkdir("data")

train.to_csv("data/train.csv", index=False)
test.to_csv("data/test.csv", index=False)
dev.to_csv("data/dev.csv", index=False)

# 3. AzureML Setup

Now, we set up the necessary components for running this as an AzureML experiment
1. Create or link to an existing `Workspace`
2. Set up an `Experiment` with `logging`
3. Create or attach existing `AmlCompute`
4. Upload our data to a `Datastore`

## 3.1 Link to or create a Workspace

First, go through the [Configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`. This will create a config.json file containing the values needed below to create a workspace.

**Note**: you do not need to fill in these values if you have a config.json in the same folder as this notebook

In [10]:
#10
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>",
)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


In [11]:
#11
print(
    "Workspace name: " + ws.name,
    "Azure region: " + ws.location,
    "Subscription id: " + ws.subscription_id,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

Workspace name: MAIDAPTest
Azure region: eastus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: nlprg


## 3.2 Set up an Experiment and Logging

In [12]:
#12
# Make a folder for the project
project_folder = "./automl-sentence-similarity"
os.makedirs(project_folder, exist_ok=True)

# Set up an experiment
experiment_name = "automl-sentence-similarity"
experiment = Experiment(ws, experiment_name)

# Add logging to our experiment
run = experiment.start_logging()

## 3.3 Link AmlCompute Compute Target

To use AzureML Pipelines we need to link a compute target as they can not be run locally (see [compute options](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-set-up-training-targets#supported-compute-targets) for explanation of the different options). We will use an AmlCompute target in this example.

In [13]:
#13
# choose your cluster
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC6", max_nodes=4
    )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute.
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-06-26T03:24:01.948000+00:00', 'errors': None, 'creationTime': '2019-06-20T23:37:54.322324+00:00', 'modifiedTime': '2019-06-20T23:38:26.084645+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


## 3.4 Upload data to Datastore

This step uploads our local data to a `Datastore` so that the data is accessible from the remote compute target and creates a `DataReference` to point to the location of the data on the Datastore. A DataStore is backed either by a Azure File Storage (default option) or Azure Blob Storage ([how to decide between these options](https://docs.microsoft.com/en-us/azure/storage/common/storage-decide-blobs-files-disks)) and data is made accessible by mounting or copying data to the compute target. `ws.datastores` lists all options for datastores and `ds.account_name` gets the name of the datastore that can be used to find it in the Azure portal.

In [14]:
#14
# Select a specific datastore or you can call ws.get_default_datastore()
datastore_name = "workspacefilestore"
ds = ws.datastores[datastore_name]

# Upload files in data folder to the datastore
ds.upload(
    src_dir="./data",
    target_path="stsbenchmark_data",
    overwrite=True,
    show_progress=True,
)

Uploading ./data\dev.csv
Uploading ./data\test.csv
Uploading ./data\train.csv
Uploaded ./data\dev.csv, 1 files out of an estimated total of 3
Uploaded ./data\train.csv, 2 files out of an estimated total of 3
Uploaded ./data\test.csv, 3 files out of an estimated total of 3


$AZUREML_DATAREFERENCE_f9d830cb0ba04033a4925f84ed15c769

We also set up a `DataReference` object that points to the data we just uploaded into the stsbenchmark_data folder. DataReference objects point to data that is accessible from a datastore and will be used an an input into our pipeline.

In [15]:
#15
input_data = DataReference(
    datastore=ds,
    data_reference_name="stsbenchmark",
    path_on_datastore="stsbenchmark_data/",
    overwrite=False,
)

# 4. Create AzureML Pipeline

Now we set up our pipeline which is made of two steps:  
1. `PythonScriptStep`: takes each sentence pair from the data in the `Datastore` and concatenates the Google USE embeddings for each sentence into one vector. This step saves the embedding feature matrix back to our `Datastore` and uses a `PipelineData` object to represent this intermediate data.  
2. `AutoMLStep`: takes the intermediate data produced by the previous step and passes it to an `AutoMLConfig` which performs the automatic model selection

## 4.1 Set up run configuration file

First we set up a `RunConfiguration` object which configures the execution environment for an experiment (sets up the conda dependencies, etc.)

In [16]:
#16
# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target

conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = aml.core.runconfig.DEFAULT_CPU_IMAGE

# Use conda_dependencies.yml to create a conda environment in the Docker image for execution
conda_run_config.environment.python.user_managed_dependencies = False

conda_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    pip_packages=[
        "azureml-sdk[automl]",
        "azureml-sdk",
        "azureml-dataprep",
        "azureml-train-automl==1.0.43.*",
    ],
    conda_packages=[
        "numpy",
        "py-xgboost<=0.80",
        "pandas",
        "tensorflow",
        "tensorflow-hub",
        "scikit-learn",
    ],
    pin_sdk_version=False,
)

print("run config is ready")

run config is ready


## 4.2 PythonScriptStep

`PythonScriptStep` is a step which runs a user-defined Python script ([documentation](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep?view=azure-ml-py) here). In this `PythonScriptStep`, we will convert our sentences into a numerical representation in order to use them in our machine learning model. We will embed both sentences using the Google Universal Sentence Encoder (provided by tensorflow-hub) and concatenate their representations into a $1024$-dimensional vector to use as features for AutoML.

**Google Universal Sentence Encoder:**
We'll use a popular sentence encoder called Google Universal Sentence Encoder (see [original paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf)). Google provides two pretrained models based on different design goals: a Transformer model (targets high accuracy even if this reduces model complexity) and a Deep Averaging Network model (DAN; targets efficient inference). Both models are trained on a variety of web sources (Wikipedia, news, question-answers pages, and discussion forums) and produced 512-dimensional embeddings. This notebook utilizes the Transformer-based encoding model which can be downloaded [here](https://tfhub.dev/google/universal-sentence-encoder-large/3) because of its better performance relative to the DAN model on the STS Benchmark dataset (see Table 2 in Google Research's [paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf)). The Transformer model produces sentence embeddings using the "encoding sub-graph of the transformer architecture" (original architecture introduced [here](https://arxiv.org/abs/1706.03762)). "This sub-graph uses attention to compute context aware representations of words in a sentence that take into account both the ordering and identity of all the other workds. The context aware word representations are converted to a fixed length sentence encoding vector by computing the element-wise sum of the representations at each word position." The input to the model is lowercase PTB-tokenized strings and the model is designed to be useful for multiple different tasks by using multi-task learning. More details about the model can be found in the [paper](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/46808.pdf) by Google Research.

### 4.2.1 Define python script to run

Define the script (called embed.py) that the `PythonScriptStep` will execute:

In [17]:
#17
%%writefile $project_folder/embed.py
import argparse
import os
import azureml.core
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

tf.logging.set_verbosity(tf.logging.ERROR)  # reduce logging output


def google_encoder(dataset):
    """ Function that embeds sentences using the Google Universal
    Sentence Encoder pretrained model
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    emb1: 512-dimensional representation of sentence1
    emb2: 512-dimensional representation of sentence2
    """
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    # Apply embedding model and normalize the input
    sts_encode1 = tf.nn.l2_normalize(embedding_model(sts_input1), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embedding_model(sts_input2), axis=1)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        emb1, emb2 = session.run(
            [sts_encode1, sts_encode2],
            feed_dict={
                sts_input1: dataset["sentence1"],
                sts_input2: dataset["sentence2"],
            },
        )
    return emb1, emb2


def feature_engineering(dataset):
    """Extracts embedding features from the dataset and returns
    features and target in a dataframe
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    df: pandas dataframe with embedding features
    scores: list of target variables
    """
    google_USE_emb1, google_USE_emb2 = google_encoder(dataset)
    n_google = google_USE_emb1.shape[1]  # length of the embeddings
    df = np.concatenate((google_USE_emb1, google_USE_emb2), axis=1)
    names = ["USEEmb1_" + str(i) for i in range(n_google)] + [
        "USEEmb2_" + str(i) for i in range(n_google)
    ]
    df = pd.DataFrame(df, columns=names)
    return df, dataset["score"]


def write_output(df, path, name):
    """Write dataframes to correct path"""
    os.makedirs(path, exist_ok=True)
    print("%s created" % path)
    df.to_csv(path + "/" + name, index=False)


# Parse arguments
parser = argparse.ArgumentParser()
parser.add_argument("--sentence_data", type=str)
parser.add_argument("--embedded_data", type=str)
args = parser.parse_args()

# Import the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embedding_model = hub.Module(module_url)

# Read data
train = pd.read_csv(args.sentence_data + "/train.csv")
dev = pd.read_csv(args.sentence_data + "/dev.csv")

# Get Google USE features
training_data, training_scores = feature_engineering(train)
validation_data, validation_scores = feature_engineering(dev)

# Write out training data to Datastore
write_output(training_data, args.embedded_data, "X_train.csv")
write_output(
    pd.DataFrame(training_scores, columns=["score"]), args.embedded_data, "y_train.csv"
)

# Write out validation data to Datastore
write_output(validation_data, args.embedded_data, "X_dev.csv")
write_output(
    pd.DataFrame(validation_scores, columns=["score"]), args.embedded_data, "y_dev.csv"
)

Writing ./automl-sentence-similarity/embed.py


### 4.2.2 Create PipelineData object

`PipelineData` objects represent a piece of intermediate data in a pipeline. Generally they are produced by one step (as an output) and then consumed by the next step (as an input), introducing an implicit order between steps in a pipeline. We create a PipelineData object that can represent the data produced by our first pipeline step that will be consumed by our second pipeline step.

In [20]:
#18
embedded_data = PipelineData("embedded_data", datastore=ds)

### 4.2.3 Create PythonScriptStep

This step defines the `PythonScriptStep`. We give the step a name, tell the step which python script to run (embed.py) and what directory that script is located in (source_directory). Note that the hash_paths parameter will be deprecated but currently is needed to check for any updates to the embed.py file.

We also link the compute target and run configuration that we made previously. Our input is the `DataReference` object (input_data) where our raw sentence data was uploaded and our ouput is the `PipelineData` object (embedded_data) where the embedded data produced by this step will be stored. These are also passed in as arguments so that we have access to the correct data paths.

In [21]:
#19
embed_step = PythonScriptStep(
    name="Embed",
    script_name="embed.py",
    arguments=["--embedded_data", embedded_data, "--sentence_data", input_data],
    inputs=[input_data],
    outputs=[embedded_data],
    compute_target=compute_target,
    runconfig=conda_run_config,
    hash_paths=["embed.py"],
    source_directory=project_folder,
    allow_reuse=True,
)

WARNING - Parameter 'hash_paths' will be deprecated. All files under source_directory will be hashed except files listed in .amlignore or .gitignore


## 4.3 AutoMLStep

`AutoMLStep` creates an AutoML step in a pipeline (see [documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlstep?view=azure-ml-py) and [basic example](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.ipynb)). When using AutoML on remote compute, rather than passing our data directly into the `AutoMLConfig` object as we did in the local example, we must define a get_data.py script with a get_data() function to pass as the data_script argument. This workflow can be used for both local and remote executions (see [details](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-auto-train-remote)). 



### 4.3.1 Define get_data script to load data

Define the get_data.py file and get_data() function that the `AutoMLStep` will execute to collect data. Note that we can directly access the path of the intermediate data (called embedded_data) through `os.environ['AZUREML_DATAREFERENCE_embedded_data']`. This is necessary because the AutoMLStep does not accept additional parameters like the PythonScriptStep does with `arguments`.

In [25]:
#20
%%writefile $project_folder/get_data.py

import os
import pandas as pd

# get location of the embedded_data for future use
EMBEDDED_DATA_REF = os.environ["AZUREML_DATAREFERENCE_embedded_data"]

def get_data():
    """Function needed to load data for use on remote AutoML experiments"""
    X_train = pd.read_csv(EMBEDDED_DATA_REF + "/X_train.csv")
    y_train = pd.read_csv(EMBEDDED_DATA_REF + "/y_train.csv")
    X_dev = pd.read_csv(EMBEDDED_DATA_REF + "/X_dev.csv")
    y_dev = pd.read_csv(EMBEDDED_DATA_REF + "/y_dev.csv")
    return {
        "X": X_train.values,
        "y": y_train.values.flatten(),
        "X_valid": X_dev.values,
        "y_valid": y_dev.values.flatten(),
    }

Overwriting ./automl-sentence-similarity/get_data.py


### 4.3.2 Create AutoMLConfig object

Now, we specify the parameters for the `AutoMLConfig` class:

**task**  
AutoML supports the following base learners for the regression task: Elastic Net, Light GBM, Gradient Boosting, Decision Tree, K-nearest Neighbors, LARS Lasso, Stochastic Gradient Descent, Random Forest, Extremely Randomized Trees, XGBoost, DNN Regressor, Linear Regression. In addition, AutoML also supports two kinds of ensemble methods: voting (weighted average of the output of multiple base learners) and stacking (training a second "metalearner" which uses the base algorithms' predictions to predict the target variable). Specific base learners can be included or excluded in the parameters for the AutoMLConfig class (whitelist_models and blacklist_models) and the voting/stacking ensemble options can be specified as well (enable_voting_ensemble and enable_stack_ensemble)

**preprocess**  
AutoML also has advanced preprocessing methods, eliminating the need for users to perform this manually. Data is automatically scaled and normalized but an additional parameter in the AutoMLConfig class enables the use of more advanced techniques including imputation, generating additional features, transformations, word embeddings, etc. (full list found [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-create-portal-experiments#preprocess)). Note that algorithm-specific preprocessing will be applied even if preprocess=False. 

**primary_metric**  
The regression metrics available are the following: Spearman Correlation (spearman_correlation), Normalized RMSE (normalized_root_mean_squared_error), Normalized MAE (normalized_mean_absolute_error), and R2 score (r2_score) 

**Constraints:**  
There is a cost_mode parameter to set cost prediction modes (see options [here](https://docs.microsoft.com/en-us/python/api/azureml-train-automl/azureml.train.automl.automlconfig?view=azure-ml-py)). To set constraints on time there are multiple parameters including experiment_exit_score (target score to exit the experiment after achieving), experiment_timeout_minutes (maximum amount of time for all combined iterations), and iterations (total number of different algorithm and parameter combinations to try).

In [26]:
#21
automl_config = AutoMLConfig(
    debug_log="automl_errors.log",
    path=project_folder,
    compute_target=compute_target,
    run_configuration=conda_run_config,
    data_script=project_folder
    + "/get_data.py",  # local path to script with get_data() function
    **automl_settings #where the autoML main settings are defined
)

### 4.3.3 Create AutoMLStep

Finally, we create `PipelineData` objects for the model data (our outputs) and then create the `AutoMLStep`. The `AutoMLStep` requires a `AutoMLConfig` object and we pass our intermediate data (embedded_data) in as the inputs. Again, note that the hash_paths parameter will be deprecated but currently is needed to check for any updates to the get_data.py file.

In [27]:
#22
# Create PipelineData objects for tracking AutoML metrics

metrics_data = PipelineData(
    name="metrics_data",
    datastore=ds,
    pipeline_output_name="metrics_output",
    training_output=TrainingOutput(type="Metrics"),
)
model_data = PipelineData(
    name="model_data",
    datastore=ds,
    pipeline_output_name="best_model_output",
    training_output=TrainingOutput(type="Model"),
)

In [28]:
#23
automl_step = AutoMLStep(
    name="AutoML",
    automl_config=automl_config,  # the AutoMLConfig object created previously
    inputs=[
        embedded_data
    ],  # inputs is the PipelineData that was the output of the previous pipeline step
    outputs=[
        metrics_data,
        model_data,
    ],  # PipelineData objects to reference metric and model information
    hash_paths=["get_data.py"],
    allow_reuse=True,
)

# 5. Run Pipeline

Now we set up our pipeline which requires specifying our `Workspace` and the ordering of the steps that we created (steps parameter). We submit the pipeline and inspect the run details using a RunDetails widget. For remote runs, the execution of iterations is asynchronous.

In [29]:
#24
pipeline = Pipeline(
    description="pipeline_embed_automl",  # give a name for the pipeline
    workspace=ws,
    steps=[embed_step, automl_step],
)

In [30]:
#25
pipeline_run = experiment.submit(pipeline)

Created step Embed [a0ecae01][edf55317-2f83-4bfe-a284-a04adaa0a44b], (This step will run and generate new outputs)
Created step AutoML [cf55fed9][cd9c8473-78d5-486a-8244-d997b6383bd7], (This step will run and generate new outputs)
Using data reference stsbenchmark for StepId [7f54cc80][e3340790-c54f-4147-8dd0-bcb80a9b7b46], (Consumers of this data are eligible to reuse prior runs.)
Submitted pipeline run: a161c227-cb7b-4c97-9682-d34841c2cf63


In [ ]:
#26
# Inspect the run details using the provided widget
RunDetails(pipeline_run).show()

![](https://nlpbp.blob.core.windows.net/images/pipelineWidget.PNG)

Alternatively, block until the run has completed.

In [ ]:
#27
pipeline_run.wait_for_completion(
    show_output=True
)  # show console output while run is in progress

**Cancel the Run**

Interrupting/Restarting the jupyter kernel will not properly cancel the run, which can lead to wasted compute resources. To avoid this, we recommend explicitly canceling a run with the following code:

`pipeline_run.cancel()`

# 6. Deploy Sentence Similarity Model

Deploying an Azure Machine Learning model as a web service creates a REST API. You can send data to this API and receive the prediction returned by the model.
In general, you create a webservice by deploying a model as an image to a Compute Target.

Some of the Compute Targets are: 
1. Azure Container Instance
2. Azure Kubernetes Service
3. Local web service

The general workflow for deploying a model is as follows:
1. Register a model
2. Prepare to deploy
3. Deploy the model to the compute target
4. Test the deployed model (webservice)

In this notebook we walk you through the process of creating a webservice running on Azure Kubernetes Service ([AKS](https://docs.microsoft.com/en-us/azure/aks/intro-kubernetes
)) by deploying the model as an image. AKS is good for high-scale production deployments. It provides fast response time and autoscaling of the deployed service. Cluster autoscaling is not supported through the Azure Machine Learning SDK. 

You can find more information on deploying and serving models [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where)


## 6.1 Register/Retrieve AutoML and Google Universal Sentence Encoder Models for Deployment

Registering a model means registering one or more files that make up a model. The Machine Learning models are registered in your current Aure Machine Learning Workspace. The model can either come from Azure Machine Learning or another location, such as your local machine.

See other ways to register a model [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where)

Below we show how to register a new model and also how to retrieve and register an existing model.

### Register a new automl model
Register the best AutoML model based on the pipeline results or load the saved model

In [ ]:
#28
automl_step_run = AutoMLStepRun(step_run=pipeline_run.find_step_run("AutoML")[0])
# to get the outputs
best_run, fitted_model = automl_step_run.get_output()

# to register the fitted_mode
description = "Pipeline Model"
tags = {"area": "nlp", "type": "sentencesimilarity pipelines"}
model = automl_step_run.register_model(description=description, tags=tags)
automl_model_name = automl_step_run.model_id
print(
    automl_step_run.model_id
)  # Use this id to deploy the model as a web service in Azure.

### Retrieve existing model from Azure
If you already have a best model then you can skip registering the model by just retrieving the latest version of model by providing its name

In [33]:
#29
automl_model_name = "76a6169d7f364bdbest"  # best fit model registered in the workspace
model = Model(ws, name=automl_model_name)
print("Found model with name", automl_model_name)

Found model with name 76a6169d7f364bdbest


### Register Google Universal Sentence Encoder Model
Register the Google Universal Sentence Encoder model if not already registered in your workspace

In [34]:
#30
# set location for where to download google tensorflow model
os.environ["TFHUB_CACHE_DIR"] = "./googleUSE"
# download model
hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
# register model
embedding_model = Model.register(
    model_path="googleUSE",
    model_name="googleUSEmodel",
    tags={"Model": "GoogleUSE"},
    description="Google Universal Sentence Embedding pretrained model",
    workspace=ws,
)
print("Registered googleUSEembeddings model")

Registering model googleUSEmodel
Registered googleUSEembeddings model


### Retrieve existing Google USE model from Azure

In [ ]:
#31
embedding_model = Model(ws, name="googleUSEmodel")
print("Found model with name googleUSEembeddings")

## 6.2 Create Scoring Script

In this section we show an example of an entry script, which is called from the deployed webservice. `score.py` is our entry script. The script must contain:
1. init() - This function loads the model in a global object.
2. run() - This function is used for model prediction. The inputs and outputs to `run()` typically use JSON for serialization and deserilization. 

In [35]:
#32
%%writefile score.py
import pickle
import json
import numpy as np
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import os

tf.logging.set_verbosity(tf.logging.ERROR)  # reduce logging output


def google_encoder(dataset):
    """ Function that embeds sentences using the Google Universal
    Sentence Encoder pretrained model
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    emb1: 512-dimensional representation of sentence1
    emb2: 512-dimensional representation of sentence2
    """
    global embedding_model, sess
    sts_input1 = tf.placeholder(tf.string, shape=(None))
    sts_input2 = tf.placeholder(tf.string, shape=(None))

    # Apply embedding model and normalize the input
    sts_encode1 = tf.nn.l2_normalize(embedding_model(sts_input1), axis=1)
    sts_encode2 = tf.nn.l2_normalize(embedding_model(sts_input2), axis=1)

    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    emb1, emb2 = sess.run(
        [sts_encode1, sts_encode2],
        feed_dict={sts_input1: dataset["sentence1"], sts_input2: dataset["sentence2"]},
    )
    return emb1, emb2


def feature_engineering(dataset):
    """Extracts embedding features from the dataset and returns
    features and target in a dataframe
    
    Parameters:
    ----------
    dataset: pandas dataframe with sentences and scores
    
    Returns:
    -------
    df: pandas dataframe with embedding features
    scores: list of target variables
    """
    google_USE_emb1, google_USE_emb2 = google_encoder(dataset)
    n_google = google_USE_emb1.shape[1]  # length of the embeddings
    return np.concatenate((google_USE_emb1, google_USE_emb2), axis=1)


def init():
    global model, googleUSE_dir_path
    model_path = Model.get_model_path(
        model_name="<<modelid>>"
    )  # this name is model.id of model that we want to deploy
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

    # load the path for google USE embedding model
    googleUSE_dir_path = Model.get_model_path(model_name="googleUSEmodel")
    os.environ["TFHUB_CACHE_DIR"] = googleUSE_dir_path


def run(rawdata):
    global embedding_model, sess, googleUSE_dir_path, model
    try:
        # load data and convert to dataframe
        data = json.loads(rawdata)["data"]
        data_df = pd.DataFrame(data, columns=["sentence1", "sentence2"])

        # begin a tensorflow session and load tensorhub module
        sess = tf.Session()
        embedding_model = hub.Module(
            googleUSE_dir_path + "/96e8f1d3d4d90ce86b2db128249eb8143a91db73"
        )

        # Embed sentences using Google USE model
        embedded_data = feature_engineering(data_df)
        # Predict using AutoML saved model
        result = model.predict(embedded_data)

    except Exception as e:
        result = str(e)
        sess.close()
        return json.dumps({"error": result})

    sess.close()
    return json.dumps({"result": result.tolist()})

Overwriting score.py


In [36]:
#33
# Substitute the actual model id in the script file.
script_file_name = "score.py"

with open(script_file_name, "r") as cefr:
    content = cefr.read()

with open(script_file_name, "w") as cefw:
    cefw.write(content.replace("<<modelid>>", automl_model_name))

## 6.3 Create a YAML File for the Environment

To ensure the fit results are consistent with the training results, the SDK dependency versions need to be the same as the environment that trains the model. The following cells create a file, pipeline_env.yml, which specifies the dependencies from the run.

In [37]:
#34
myenv = CondaDependencies.create(
    conda_packages=[
        "numpy",
        "scikit-learn",
        "py-xgboost<=0.80",
        "pandas",
        "tensorflow",
        "tensorflow-hub",
    ],
    pip_packages=["azureml-sdk[automl]==1.0.43.*"],
    python_version="3.6.8",
)

conda_env_file_name = "pipeline_env.yml"
myenv.save_to_file(".", conda_env_file_name)

'pipeline_env.yml'

## 6.4 Image Creation

In this step we create a container image which is wrapper containing the entry script, yaml file with package dependencies and the model. The created image is then deployed as a webservice in the next step. This step can take up to 10 minutes and even longer if the model is large.

In [38]:
#35
# trying to add dependencies
image_config = ContainerImage.image_configuration(
    execution_script=script_file_name,
    runtime="python",
    conda_file=conda_env_file_name,
    description="Image with aml pipeline model",
    tags={"area": "nlp", "type": "sentencesimilarity pipeline"},
)

image = ContainerImage.create(
    name="pipeline-image",
    # this is the model object
    models=[model, embedding_model],  # add both embedding and autoML models
    image_config=image_config,
    workspace=ws,
)

image.wait_for_creation(show_output=True)

Creating image
Running......................................................................................................................
Succeeded
Image creation operation finished for image pipeline-image:59, operation "Succeeded"


If the above step fails, then use below command to see logs.

In [ ]:
# image.get_logs()

## 6.5 Provision the AKS Cluster

**Time estimate:** Approximately 20 minutes.

Creating or attaching an AKS cluster is a one time process for your workspace. You can reuse this cluster for multiple deployments. If you delete the cluster or the resource group that contains it, you must create a new cluster the next time you need to deploy. You can have multiple AKS clusters attached to your workspace.

If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [40]:
#36
# create aks cluser

# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

# Create the cluster
aks_target = ComputeTarget.create(
    workspace=ws, name="nlp-aks1", provisioning_configuration=prov_config
)


## 6.6 Deploy the Image as a Web Service on Azure Kubernetes Service

In the case of deployment on AKS, in addition to the Docker image, we need to define computational resources. This is typically a cluster of CPUs or a cluster of GPUs. If we already have a Kubernetes-managed cluster in our workspace, we can use it, otherwise, we can create a new one.

In this notebook we will use the cluster in the above cell.

In [41]:
#37
# Set the web service configuration
aks_config = AksWebservice.deploy_configuration()

We are now ready to deploy our web service. We will deploy from the Docker image. It contains our AutoML model as well as the Google Universal Sentence Encoder model and the conda environment needed for the scoring script to work properly. The parameters to pass to the Webservice.deploy_from_image() command are similar to those used for the deployment on ACI. The only major difference is the compute target (aks_target), i.e. the CPU cluster we just spun up.

**Note:** This deployment takes a few minutes to complete.

In [44]:
#38
# deploy image as web service
aks_service_name = "aks-pipelines-service"

aks_service = Webservice.deploy_from_image(
    workspace=ws,
    name=aks_service_name,
    image=image,
    deployment_config=aks_config,
    deployment_target=aks_target,
)
aks_service.wait_for_deployment(show_output=True)
print(aks_service.state)

Creating service
Running....................................
SucceededAKS service creation operation finished, operation "Succeeded"
Healthy


If the above step fails then use below command to see logs

In [ ]:
# aks_service.get_logs()

## 6.7 Test Deployed Webservice

Testing the deployed model means running the created webservice. <br>
The deployed model can be tested by passing a list of sentence pairs. The output will be a score between 0 and 5, with 0 indicating no meaning overlap between the sentences and 5 meaning equivalence.

The run method expects input in json format. The Run() method retrieves API keys behind the scenes to make sure that the call is authenticated. The service has a timeout (default of ~30 seconds) which does not allow passing the large test dataset. To overcome this, you can batch data and send it to the service.

In [45]:
#39
sentences = [
    ["This is sentence1", "This is sentence1"],
    ["A hungry cat.", "A sleeping cat"],
    ["Its summer time ", "Winter is coming"],
]
data = {"data": sentences}
data = json.dumps(data)

In [46]:
#40
# Set up a Timer to see how long the model takes to predict
t = Timer()

t.start()
score = aks_service.run(input_data=data)
t.stop()

print("Time elapsed: {}".format(t))

result = json.loads(score)
try:
    output = result["result"]
    print("Number of samples predicted: {}".format(len(output)))
    print(output)
except:
    print(result["error"])

Time elapsed: 10.8916
Number of samples predicted: 3
[4.522378114490067, 2.333906792348142, 2.502971119054824]


Finally, we'll calculate the Pearson Correlation on the test set.

**What is Pearson Correlation?**

Our evaluation metric is Pearson correlation ($\rho$) which is a measure of the linear correlation between two variables. The formula for calculating Pearson correlation is as follows:  

$$\rho_{X,Y} = \frac{E[(X-\mu_X)(Y-\mu_Y)]}{\sigma_X \sigma_Y}$$

This metric takes a value in [-1,1] where -1 represents a perfect negative correlation, 1 represents a perfect positive correlation, and 0 represents no correlation. We utilize the Pearson correlation metric as this is the main metric that [SentEval](http://nlpprogress.com/english/semantic_textual_similarity.html), a widely-used evaluation toolkit for evaluation sentence representations, uses for the STS Benchmark dataset.

In [51]:
#41
# load test set sentences
data = pd.read_csv("data/test.csv")
train_y = data["score"].values.flatten()
train_x = data.drop("score", axis=1).values.tolist()
data = {"data": train_x[:500]}
data = json.dumps(data)

In [53]:
#42
# Set up a Timer to see how long the model takes to predict
t = Timer()

t.start()
score = aks_service.run(input_data=data)
t.stop()

print("Time elapsed: {}".format(t))

result = json.loads(score)

try:
    output = result["result"]
    print("Number of sample predicted : {}".format(len(output)))
    print(output)
except:
    print(result["error"])

Time elapsed: 17.8458
Number of sample predicted : 500
[1.778579367510987, 3.50046288707411, 3.2538821258681896, 4.069305058515004, 1.8320877577432195, 1.954834972650947, 3.5939983298935054, 1.897463629391319, 2.5197130778016654, 1.8823957738334811, 1.8823957738334811, 3.6895210831755922, 1.8599267741539272, 3.1787909087040562, 2.7087431329327916, 1.5672046737532321, 4.451727638080559, 3.6796594501884523, 3.544795325159195, 1.9030900379648785, 2.8150889983631284, 2.183746733671711, 3.601602614164534, 4.206702507830564, 1.6307750689271787, 3.234565416414611, 1.867143843466714, 3.4361162329783883, 1.8026588109513297, 2.9688938044152415, 3.8929035816615896, 3.219924328188107, 2.0385247308394234, 3.798599603279599, 2.829788590045488, -0.3670312508170009, 1.8917018009786544, 2.5162131407684174, 2.5313683231655797, 0.7973264807516905, 3.262854998024572, 3.445451775581639, 3.313722165367336, 1.493230130992738, 2.5124437403792377, 1.3942647183703063, 1.2066715622591626, -0.028170077204135217, 

In [54]:
#43
# get Pearson Correlation
print(pearsonr(output, train_y[:500])[0])

0.8200285709168692
